<a href="https://colab.research.google.com/github/kamat-v/HF_Transformers_Notebooks/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

In [ ]:
!unzip -nq spa-eng.zip

In [ ]:
!ls

In [ ]:
ls spa-eng

In [ ]:
!head spa-eng/spa.txt

In [29]:
eng2spa={}
for line in open('spa-eng/spa.txt', encoding='utf-8'):
  line=line.rstrip()
  eng, spa = line.split('\t')
  if eng not in eng2spa:
    eng2spa[eng]=[]
  eng2spa[eng].append(spa)

In [ ]:
eng2spa

In [10]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [11]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
smoother = SmoothingFunction()

In [ ]:
tokens = tokenizer.tokenize('¿Qué me cuentas?'.lower())
sentence_bleu([tokens], tokens)

In [ ]:
sentence_bleu([['hi']],['hi'])

In [ ]:
tokens = tokenizer.tokenize('¿Qué me cuentas?'.lower())
sentence_bleu([tokens], tokens, smoothing_function=smoother.method4)

In [ ]:
sentence_bleu([['hi']], ['hi'])

In [30]:
eng2spa_tokens={}
for eng, spa_list in eng2spa.items():
  spa_list_tokens=[]
  for text in spa_list:
    tokens=tokenizer.tokenize(text.lower())
    spa_list_tokens.append(tokens)
  eng2spa_tokens[eng]=spa_list_tokens

In [ ]:
!pip install transformers sentencepiece transformers[sentencepiece]

In [ ]:
from transformers import pipeline
translator = pipeline("translation",
                      model='Helsinki-NLP/opus-mt-en-es', device=0)

In [ ]:
translator("I like eggs and ham")

In [ ]:
eng_phrases=list(eng2spa.keys())
len(eng_phrases)

In [24]:
eng_phrases_subset=eng_phrases[20_000:20_005]

In [25]:
translations=translator(eng_phrases_subset)

In [ ]:
translations[0]

In [ ]:
scores=[]
for eng, pred in zip(eng_phrases_subset, translations):
  matches=eng2spa_tokens[eng]
  spa_pred=tokenizer.tokenize(pred['translation_text'].lower())
  score=sentence_bleu(matches, spa_pred)
  scores.append(score)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(scores, bins=50)

In [33]:
import numpy as np
np.mean(scores)

np.float64(0.6)

In [36]:
np.random.seed(1)

def print_random_translation():
  i=np.random.choice(len(eng_phrases_subset))
  eng=eng_phrases_subset[i]
  print("ENG:", eng)

  translation=translations[i]['translation_text']
  print("ESP:", translation)

  matches=eng2spa[eng]
  print("Matches:", matches)

In [ ]:
print_random_translation()

In [ ]:
print_random_translation()

![](https://deeplearningcourses.com/notebooks_v3_pxl?sc=8Jty5lhP77FmKx1Scgr7YA&n=Pipeline+Translation)